In [0]:
import requests
import pandas as pd
import json
import os 
from bs4 import BeautifulSoup as BS
from urllib.parse import urljoin

In [0]:
url = "https://www.nassnig.org/mps/get_legislators/?chamber="
single_url = "https://www.nassnig.org/mps/single/"

## gather details via JSON

In [0]:
members_list = []
for i in range(1,3,1):
  link = url + str(i)
  response = requests.get(link).json()
  members = response['data']
  for row in members:
    member = {"name" : row[0],
              "state" : row[1],
              "party" :row[3],
              "area": row[2],
              "member_id" : row[-1],
              'url' :  single_url + row[-1]
              }
     
    # print(member['url']) 
    members_list.append(member)
 
        

In [0]:
len(members_list)

460

## Use Beautiful Soup for NG members

In [0]:
domain = "https://www.nassnig.org"
fetch_bills = "https://www.nassnig.org/mps/fetch_mp_bills/?sponsor="

m_Data = []
house_rep = []
for member in members_list:
  # house_rep = {}
  member_url = member['url']
  id_code = member['member_id']
  bills_url = fetch_bills + str(id_code)
  # print(member_url)
  response = requests.get(member_url)
  c = response.content
  soup = BS(c,'html.parser')
  member_data = {
      'name': soup.find('h3').text.split(".")[1].strip(),
      'alias': member['name'],
      'dob': soup.find_all('a',{'class':'list-group-item'})[0].text.split(':')[1].strip(),
      'image': urljoin(domain,soup.find('img',{'alt':'Avatar'}).get('src')),
      'address': soup.find('div',{'class':'col-md-3'}).find_all('a')[4].text.split(':')[1].strip().title(),
      'phone': soup.find('div',{'class':'col-md-3'}).find_all('a')[5].text.split(':')[1].strip(),
      'email': soup.find('div',{'class':'col-md-3'}).find_all('a')[6].text.split(':')[1].strip(),
      'previous_office': soup.find_all('div',{'class':'tab-content'})[1].text.strip().replace("\xa0","").split('\n'),
      'education': soup.find('div',{'id':'tab-education'}).text.strip().split("                 "),
      'awards': soup.find('div',{'id':'tab-awards'}).text.strip().split("                 "),
      'chamber': soup.find_all('a',{'class':'list-group-item'})[1].text.split(':')[1].strip(),
      'party': soup.find_all('a',{'class':'list-group-item'})[2].text.split(':')[1].strip(),
      'position': soup.find_all('a',{'class':'list-group-item'})[3].text.strip(),
      'state': member['state'],
      'region': soup.find('div',{'class':'heading-block'}).span.text.strip(),
      'area': member['area'],
      'source': member['url']

  }
  try:
    response = requests.get(bills_url).json()
    bills = response['data']
    member_data['bills'] = bills
  except:
    print("no bills found")
    pass

  house_rep.append(member_data)

  # df = pd.DataFrame(data)

## Work from DataFrame

In [0]:
df = pd.DataFrame(house_rep)

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             460 non-null    object
 1   alias            460 non-null    object
 2   dob              460 non-null    object
 3   image            460 non-null    object
 4   address          460 non-null    object
 5   phone            460 non-null    object
 6   email            460 non-null    object
 7   previous_office  460 non-null    object
 8   education        460 non-null    object
 9   awards           460 non-null    object
 10  chamber          460 non-null    object
 11  party            460 non-null    object
 12  position         460 non-null    object
 13  state            460 non-null    object
 14  region           460 non-null    object
 15  area             460 non-null    object
 16  source           460 non-null    object
 17  bills            460 non-null    ob

In [0]:
# df.to_csv('NG_NA_MPs_Senators.csv')